***
## Gradient of Episodic Memory - GEM

### Benchmark: Split CIFAR10

Experiment reproducing **Gradient of Episodic Memory** method:

The main future of GEM is an *episodic memory* M<sub>t</sub>, which stores a subset of the observed examples from task *t*. Focus on minimizing negative backward transfer by the efficient use of episodic memory.

`References:`
- Gradient of Episodic Memory: https://arxiv.org/abs/1706.08840 

**Gradient of Episodic Memory strategy:**
https://avalanche-api.continualai.org/en/v0.1.0/generated/avalanche.training.GEM.html
***

In [ ]:
# Testing framework and test runner
import unittest

import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD

# Avalanche library from ContinualAI
import avalanche

# Models and benchmarks
from avalanche.models import SimpleMLP
from avalanche.benchmarks.classic  import SplitCIFAR10

# Loggers
from avalanche.logging import InteractiveLogger, TensorboardLogger

# Evaluation
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics, timing_metrics, forgetting_metrics

# Extras: Model and utils
from utils import arguments, get_average_metric, get_target_result

## LwF technique
class GEM(unittest.TestCase): #TestCase class

    #### Split CIFAR10 benchmark
    def test_scifar10(self, override_args=None):
        
        scenario = SplitCIFAR10(n_experiences=5, return_task_id=False)
               
        # --- Strategy instantiation --- # 
        # 1. Model
        # 2. Optimizer
        # 3. Loss function
        
        # -> ADDITIONAL ARGUMENTS allow to customize training
        args = arguments({ 'cuda': 0,               # GPU
                           'patterns_per_exp': 256, # Patterns per experience in the memory
                           'memory_strength' : 0.5, # Offset to the projection direction to favour BWT 
                           'learning_rate': 0.1,    # Learning rate
                           'train_epochs' : 1,      # Training epochs
                           'eval_mb_size' : 128,    # Evaluation minibatch size
                           'train_mb_size': 10}, override_args) # Train minibatch size

        # Set up and run CUDA operations,
        # if CUDA is available, utilize GPUs for computation.
        device = torch.device(f"cuda:{args.cuda}"
                              if torch.cuda.is_available() and args.cuda >= 0 
                              else "cpu")
        
        model = SimpleMLP(num_classes=10, input_size=32*32*3, hidden_size=100, hidden_layers=1, drop_rate=0)

        optimizer = SGD(model.parameters(), lr=args.learning_rate)
        criterion = CrossEntropyLoss()
        
        # ------------------------ LOG ------------------------ #
        # logging results over-time to examine the experiment in real-time
        loggers = []
        
        # log to Tensorboard
        loggers.append(TensorboardLogger())
        
        # Avalanche logging module, displays a progress bar during training and evaluation
        interactive_logger = avalanche.logging.InteractiveLogger()
        
        # -------------------- EVALUATION -------------------- #
        # Metrics of main interest to be tracked
        eval_plugin = EvaluationPlugin(
            accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
            #timing_metrics(epoch=True),
            #forgetting_metrics(experience=True, stream=True),
            #loggers=[InteractiveLogger()],
            loggers   = loggers,
            benchmark = scenario
        )

        # -> CONTINUAL LEARNING STRATEGY: GEM
        cl_strategy = avalanche.training.GEM(model, optimizer, criterion,
                                             # additional arguments
                                             patterns_per_exp = args.patterns_per_exp, 
                                             memory_strength  = args.memory_strength,
                                             train_mb_size    = args.train_mb_size, 
                                             train_epochs     = args.train_epochs,
                                             device = device,
                                             # evaluation
                                             evaluator = eval_plugin,
        )
        
        # --- Training loop --- #
        print('Starting experiment...')
        
        for experience in scenario.train_stream:
            print('Current experience {}, contains: {} patterns'.format(experience.current_experience, len(experience.dataset)))
            print('Current classes: ',experience.classes_in_this_experience)
            
            # Train
            cl_strategy.train(experience)
            print('Training completed')
            
            # Accuracy over the whole test set (no access to task-ID at inference time)
            print('Computing accuracy over the whole test set')
            res = cl_strategy.eval(scenario.test_stream)
            
        
        avg_stream_acc = get_average_metric(res)
        print(f"GEM-SplitCIFAR10 Average Stream Accuracy: {avg_stream_acc:.2f}")

### Run and Evaluate the experiment
- Create an instance of the strategy object
- Execute the strategy on a benchmark

In [ ]:
# Create the strategy
s = GEM()

# Run the experiment with custom parameters
s.test_scifar10()
